In [1]:
import tensorflow as tf
import keras
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense, Dropout, BatchNormalization
from keras.models import Sequential
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator, DirectoryIterator
#to display heatmaps
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [2]:
import keras
from keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D
from keras.layers import Flatten, Dense, Dropout, BatchNormalization, Reshape, Permute, Multiply, Input, Activation
from keras.models import Sequential, Model
import tensorflow.keras.backend as K

def SqueezeExcite(x, ratio=16, name=''):
    nb_chan = K.int_shape(x)[-1]

    y = GlobalAveragePooling2D(name='SE_avg_{}'.format(name))(x)
    y = Dense(nb_chan // ratio, activation='relu', name='se_dense1_{}'.format(name))(y)
    y = Dense(nb_chan, activation='sigmoid', name='dense2_{}'.format(name))(y)

    y = Multiply(name='se_mul_{}'.format(name))([x, y])
    return y

def SE_Conv2D_block(x, filters, kernel_size, name):
    y = Conv2D(filters, kernel_size, padding='same', name='conv_{}'.format(name))(x)
    #squeeze excite in this
    y = SqueezeExcite(y, ratio=16, name=name)
    #other relu and maxpools
    y = BatchNormalization(name='BatchNorm_{}'.format(name))(y)
    y = Activation('relu', name='Activation_{}'.format(name))(y)
    y = MaxPool2D(pool_size=(2, 2), padding='same', name='{}_pool'.format(name))(y)
    y = Dropout(0.25)(y)
    return y


def SE_Dense_block(x, size, name, bn=True):
    y = Dense(size, name='dense_{}'.format(name))(x)
    if bn:
        y = BatchNormalization(name='BatchNorm_{}'.format(name))(y)
    y = Activation('relu', name='Activation_{}'.format(name))(y)
    return y

In [3]:
#huge thanks to -->https://github.com/arthurdouillard/keras-squeeze_and_excitation_network


def SE_Model(nb_class, input_shape, include_top=True, weights = False):
    """SE net without the splitted stream."""

    img_input = Input(shape=input_shape)

    x = SE_Conv2D_block(img_input, 32, (3, 3), name='block1')
    

    params = [
        (64, (3, 3)),
        (128, (3, 3)),
        (256, (3, 3))
    ]

    for i, (filters, kernel_size) in enumerate(params, start=2):
        x = SE_Conv2D_block(x, filters, kernel_size, name='block{}'.format(i))

    if include_top:
        x = Flatten(name='flatten')(x)
#         x = SE_Dense_block(x, 512, 'top1', bn=True)
#         x = SE_Dense_block(x, 128, 'top2', bn=True)
#         x = SE_Dense_block(x, 32, 'top3', bn=True)
        x = Dense(nb_class, activation='sigmoid')(x)

    model = Model(inputs=img_input, outputs=x)

    if weights:
        print('Loading')
        model.load_weights(weights)
        print("Weight Loaded")

    return model

model = SE_Model(nb_class=1, input_shape=(128, 128, 3), weights='SEModelWeights.h5')
model.summary()
# model.compile()

optimizer = keras.optimizers.Adam(lr=0.00001)
loss_fn = keras.losses.BinaryCrossentropy(from_logits=True) #from_logits=True means output probabilities are not normalized
acc_metric = keras.metrics.BinaryAccuracy()
val_acc_metric = keras.metrics.BinaryAccuracy()
val_loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)

Loading
Weight Loaded
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv_block1 (Conv2D)            (None, 128, 128, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
SE_avg_block1 (GlobalAveragePoo (None, 32)           0           conv_block1[0][0]                
__________________________________________________________________________________________________
se_dense1_block1 (Dense)        (None, 2)            66          SE_avg_block1[0][0]              
________________________________________________________________________

In [4]:
@tf.function
def test_step(x):
    val_preds = model(x, training=False)
#     Update val metrics
#     val_acc_metric.update_state(y, val_preds)
    return val_preds

In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'imgs/',
    target_size=(128, 128),
    batch_size=13,
    shuffle=True,
    class_mode='binary')

recent_test_batch = test_generator.next()

x_test_batch = recent_test_batch[0]
y_test_batch = recent_test_batch[1]
# print(x_test_batch)
# print(y_test_batch)

pred = test_step(x_test_batch)
# print(val_acc_metric.result())
# val_acc_metric.reset_states()
# print(pred)
for i in range(13):
    print(y_test_batch[i])
    print(np.reshape(tf.get_static_value(pred),(1,13))[0][i])


Found 16 images belonging to 2 classes.
0.0
0.13743323
1.0
0.0037907644
1.0
0.005911629
0.0
2.4289534e-08
0.0
0.13069552
1.0
0.11173201
1.0
0.9996624
1.0
0.021168925
0.0
0.98164946
0.0
6.4619708e-06
0.0
0.99884087
0.0
0.9815214
0.0
0.001461906


In [6]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array



def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [11]:
# def put_text(imgPath, actual, pred):
#     img = cv2.imread(imgPath)
#     actual = "Expected: "+str(actual)
#     pred = "Predicted: "+str(format(pred,".3f"))
#     img =cv2.putText(img, str(actual), (0, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 2)
#     img =cv2.putText(img, str(pred), (0, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 2)
#     cv2.imwrite(imgPath, img)

def put_text(imgPath, actual, pred):
    fig, ax = plt.subplots()
    ax.set_xlim(0, 100)
    ax.set_ylim(-30, 100)
    
    img = mpimg.imread(imgPath)
    img = OffsetImage(img, zoom=0.4)
    ab = AnnotationBbox(img, (50, 60))
    ax.add_artist(ab)
    
    plt.text(11, -8, "Ground Truth: {}".format(imgPath.split('/')[1][0:-5]), fontsize = 14)
    plt.axis('off')
    
    drowsy = (1 - pred)*100
    
#     Rectangle(xy, width, height
    plt.text(11, -18, "Predicted: {:.2f}% drowsy".format(drowsy), fontsize = 14)
    size = 40
    rect1 = matplotlib.patches.Rectangle((12,-20),size*(1-pred),-5, color = 'red')
    rect2 = matplotlib.patches.Rectangle((12+(size*(1-pred)),-20),size*pred,-5, color = 'green')
    
    greenRect = matplotlib.patches.Rectangle((70,-5),2,2, color = 'green')
    plt.text(74, -5, ": Non-drowsy", fontsize=8)
    redRect = matplotlib.patches.Rectangle((70,-12),2,2, color = 'red')
    plt.text(74,-12, ": Drowsy", fontsize = 8)


    ax.add_patch(rect1)
    ax.add_patch(rect2)
    ax.add_patch(greenRect)
    ax.add_patch(redRect)
    
    plt.savefig(imgPath)

    plt.close()

In [8]:
def save_and_display_gradcam(img_path,actual,pred, heatmap, alpha=0.4):
    # Load the original image
    img = keras.preprocessing.image.load_img(img_path)
    img = keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    imgPath = img_path.partition('/')[2].partition('\\')[2]
    imgPath = "SEResults/"+imgPath
    superimposed_img.save(imgPath)
    superimposed_img = put_text(imgPath, actual,pred)
    

    # Display Grad CAM
#     display(Image(cam_path))

In [9]:
class ImageWithNames(DirectoryIterator):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.filenames_np = np.array(self.filepaths)
        self.class_mode = None # so that we only get the images back

    def _get_batches_of_transformed_samples(self, index_array):
#         print(super()._get_batches_of_transformed_samples(index_array))
        return (super()._get_batches_of_transformed_samples(index_array),
                self.filenames_np[index_array])

In [12]:
# Prepare image
# img_array = preprocess_input(get_img_array("imgs/drowsy/drowsy1", size=(1,128,128,3)))

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = ImageWithNames(
    'imgs/',test_datagen,
    target_size=(128, 128),
    batch_size=16,
#     shuffle=True,
    class_mode='binary')
x_test_batch, fileNames = next(test_generator)
# print(recent_test_batch)
# x_test_batch = recent_test_batch[0]
# y_test_batch = recent_test_batch[1]

for i in range(len(x_test_batch)):
#     print(np.resize(x_test_batch[i],(128,128,3)).shape)
#     print(fileNames)
    # Print what the top predicted class is
    preds = test_step(np.expand_dims(x_test_batch[i],0))
    pred = np.reshape(tf.get_static_value(preds),(1,1))[0][0]
    print(pred)
    
    
    # Remove last layer's softmax
    model.layers[-1].activation = None
    
    # Generate class activation heatmap
#     img = np.resize(x_test_batch[i],(128,128,3))*255.0
    heatmap = make_gradcam_heatmap(tf.expand_dims(x_test_batch[i],0), model, 'conv_block4')

    # Display heatmap
#     plt.matshow(heatmap)
#     plt.show()
    print(fileNames[i])
    actual = '0.0'
    if 'non-drowsy' in fileNames[i]:
        actual = '1.0'
    save_and_display_gradcam(fileNames[i],actual, pred, heatmap)
    
# val_acc = val_acc_metric.result()
# val_acc_metric.reset_states()
# print("overall accuracy: ", val_acc )

Found 16 images belonging to 2 classes.
0.0014619074
imgs/drowsy\drowsy6.jpg
0.9815214
imgs/drowsy\drowsy4.jpg
0.11173186
imgs/non-drowsy\non-drowsy4.jpg
0.0037907842
imgs/non-drowsy\non-drowsy5.jpg
0.9770692
imgs/non-drowsy\non-drowsy6.jpg
0.021168768
imgs/non-drowsy\non-drowsy7.jpg
0.9996624
imgs/non-drowsy\non-drowsy1.jpg
0.99884087
imgs/drowsy\drowsy1.jpg
0.98164946
imgs/drowsy\drowsy7.jpg
0.40943506
imgs/non-drowsy\non-drowsy2.jpg
0.9693192
imgs/non-drowsy\non-drowsy8.jpg
6.4619585e-06
imgs/drowsy\drowsy8.jpg
0.13069558
imgs/drowsy\drowsy3.jpg
2.4289626e-08
imgs/drowsy\drowsy5.jpg
0.13743338
imgs/drowsy\drowsy2.jpg
0.005911584
imgs/non-drowsy\non-drowsy3.jpg
